In [1]:
import pandas as pd
import MeCab
import gensim
import numpy as np
import matplotlib.pyplot as plt
import ipadic
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

c:\Users\higak\anaconda3\envs\nlp\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
nega = pd.read_csv("./out/nega.csv")
print(nega.head())

                                             content  negaposi
0  コロナワクチンは、人体をスパイクタンパク質工場に変化させ、さらに摂取していない人にも被爆させ... -0.200000
1  「マスク意味ないですよね？コロナより熱中症の方が怖いのでしません」と答えると、\n\n「ルー... -0.600000
2       @w0tteNiPzcSbr0V その時はコロナはありましたっけ?\nコロナでの負担が。 -0.333333
3  @kbc_sunshine くまざわ書店 サンシャインシティアルパ店のみんな、今「えいが　そ... -0.142857
4  今朝から発熱報告が続々。\nこれから何人がコロナ陽性になるのか。\nかなり厳しい状況になって... -0.250000


In [3]:
def parse(tweet_temp):
    t = MeCab.Tagger(ipadic.MECAB_ARGS)
    temp1 = t.parse(tweet_temp)
    temp2 = temp1.split("\n")
    t_list = []
    for keitaiso in temp2:
        if keitaiso not in ["EOS", ""]:
            word, hinshi = keitaiso.split("\t")
            t_temp = [word] + hinshi.split(",")
            if len(t_temp) != 10:
                t_temp += ["*"]*(10-len(t_temp))
            t_list.append(t_temp)
    return t_list

def parse_to_df(tweet_temp):
    return pd.DataFrame(parse(tweet_temp),
                        columns=["単語","品詞","品詞細分類1",
                                 "品詞細分類2","品詞細分類3",
                                 "活用型","活用形","原形","読み","発音"])
    
def make_lda_docs(texts):
    docs = []
    for text in texts:
        df = parse_to_df(text)
        extract_df = df[(df["品詞"]+"/"+df["品詞細分類1"]).isin(["名詞/一般","名詞/固有名詞"])]
        extract_df = extract_df[extract_df["原形"]!="*"]
        doc = []
        for genkei in extract_df["原形"]:
            doc.append(genkei)
        docs.append(doc)
    return docs

In [4]:
texts = nega["content"].values
docs = make_lda_docs(texts)
print(docs[0])
dictionary = gensim.corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

['コロナ', 'ワクチン', '人体', 'タンパク質', '工場', '人']


In [5]:
print("テキスト：")
print(texts[0][:200])
print(" ")
print("LDAで読む単語：")
print(",".join(docs[0][:50]))

テキスト：
コロナワクチンは、人体をスパイクタンパク質工場に変化させ、さらに摂取していない人にも被爆させる
 https://t.co/f7NtlMYRoX
 
LDAで読む単語：
コロナ,ワクチン,人体,タンパク質,工場,人


In [6]:
n_cluster = 4
lda = gensim.models.LdaModel(
                corpus=corpus,
                id2word=dictionary,
                num_topics=n_cluster,
                minimum_probability=0.001,
                passes=20,
                update_every=0,
                chunksize=10000,
                random_state=1
)

In [7]:
corpus_lda = lda[corpus]
arr = gensim.matutils.corpus2dense(
        corpus_lda,
        num_terms=n_cluster
).T

### トピック-単語分布の可視化

In [8]:
lists = []
for i in range(n_cluster):
    temp_df = pd.DataFrame(lda.show_topic(i), columns=["word", "score"])
    temp_df["topic"] = i
    lists.append(temp_df)
topic_word_df = pd.concat(lists, ignore_index=True)

topic_word_df.to_csv("./out/topic_word_df.csv", index=False)

In [9]:
#topic_word_df["rank"] = topic_word_df.groupby("topic")["score"].rank()
#topic_word_df.pivot(index='topic', columns='rank', values='word')

In [10]:
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, "./out/pyldavis_output.html")

c:\Users\higak\anaconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
